In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import widgets
from IPython.display import display

import gym

from matplotlib import animation
from JSAnimation.IPython_display import display_animation
from time import gmtime, strftime
import random

import tensorflow as tf


def leftclicked(something):
    onclick(3)

def rightclicked(something):
    onclick(2)

def firstclicked(something):
    onclick(0)

def secondclicked(something):
    onclick(1)
    
def display_buttons():
    """ Display the buttons you can use to apply a force to the cart """
    left = widgets.Button(description="<")
    right = widgets.Button(description=">")
    one = widgets.Button(description="1")
    two = widgets.Button(description="2")
    display(left, right, one, two)
    
    left.on_click(leftclicked)
    right.on_click(rightclicked)
    one.on_click(firstclicked)
    two.on_click(secondclicked)

# Create the environment and display the initial state
env = gym.make('Breakout-v0')
observation = env.reset()
firstframe = env.render(mode = 'rgb_array')
fig,ax = plt.subplots()
im = ax.imshow(firstframe) 

# Show the buttons to control the cart
display_buttons()


# Function that defines what happens when you click one of the buttons
frames = []
def onclick(action):
    global frames
    observation, reward, done, info = env.step(action)
    #print(info)
    frame = env.render(mode = 'rgb_array')
    im.set_data(frame)
    frames.append(frame)

    

[2017-06-30 12:11:01,499] Making new env: Breakout-v0


<IPython.core.display.Javascript object>

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [2]:
print(env)

<TimeLimit<AtariEnv<Breakout-v0>>>


In [3]:
def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    if len(frames) == 0:
        return
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))
    
display_frames_as_gif(frames)

In [4]:
def rgb2gray(rgb):
    rgb = rgb[::2, ::2] # subsample
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

history = [] # (observation, action, reward, done)
RANDOM_INIT_COUNT = 10000

for _ in xrange(RANDOM_INIT_COUNT):
    action = env.action_space.sample() 
    newobservation, reward, done, info = env.step(action)
    observation = rgb2gray(np.array(observation))
    history.append((observation, action, reward, done))
    observation = newobservation
    if done:
        observation = env.reset()
        

            

In [5]:
def calc_future_reward(history, startindex, discountfactor):
    factor = 1.0
    futurereward = 0.0
    for index in range(startindex, len(history)):
        reward = history[index][2]
        done = history[index][3]
        futurereward += factor * reward
        factor *= discountfactor
        if done: 
            break
    return futurereward

gamma = 0.99        
    
def frames_to_nd_image(frames):
    imageh, imagew = frames[0].shape
    newimage = np.zeros((imageh, imagew,len(frames)))
    for index, frame in enumerate(frames):
        newimage[:,:,index] = frame
    return newimage


state_rewards = [] # (state, action, reward, state_s+1)
FRAMES_TO_CONSIDER = 4
ACTIONCOUNT = 4
IMAGE_HEIGHT = 105
IMAGE_WIDTH = 80

# INIT STATE!
oneimage = env.reset()
oneimage = rgb2gray(oneimage)
frames = [oneimage]*FRAMES_TO_CONSIDER
state = frames_to_nd_image(frames)
for _ in range(100):
    state_rewards.append((state,0,0,state))
    

In [6]:

inputimage = tf.placeholder(tf.float32, [None, IMAGE_HEIGHT, IMAGE_WIDTH, FRAMES_TO_CONSIDER])
reward = tf.placeholder(tf.float32, [None])
action = tf.placeholder(tf.float32, [None, ACTIONCOUNT])

def conv_layer(input_image,ksize,in_channels,out_channels,stride,scope_name,activation_function=tf.nn.relu):
    with tf.variable_scope(scope_name):
        filter = tf.Variable(tf.random_normal([ksize,ksize,in_channels,out_channels],stddev=0.003))
        output = tf.nn.conv2d(input_image,filter, strides=[1, stride, stride, 1], padding='SAME')
        if activation_function:
            output = activation_function(output)
        return output, filter
conv1 , _ = conv_layer(inputimage, 8, FRAMES_TO_CONSIDER, 16, 4, 'conv1')
conv2 , _ = conv_layer(conv1, 4, 16, 32, 2,'conv2')
conv3 , _ = conv_layer(conv2, 4, 32, 32, 2,'conv3')
print(conv3)
indixes = conv3.get_shape().as_list()[1:]
feature_count = indixes[0]*indixes[1]*indixes[2]
reshaped = tf.reshape(conv3, [-1, feature_count])
print(reshaped)
lin1_weights = tf.Variable(tf.random_normal([feature_count, 256],stddev=0.0001))
lin2_weights = tf.Variable(tf.random_normal([256, ACTIONCOUNT],stddev=0.001))
lin1_bias = tf.Variable(tf.zeros([256]))
lin2_bias = tf.Variable(tf.zeros([ACTIONCOUNT]))

linear1 = tf.nn.sigmoid(tf.add(tf.matmul(reshaped,lin1_weights),lin1_bias))
output = tf.nn.relu(tf.nn.sigmoid(tf.add(tf.matmul(linear1,lin2_weights),lin2_bias)))
#output = tf.contrib.layers.fully_connected(linear1, ACTIONCOUNT, activation_fn = None)
print(output)
qreward = tf.reduce_sum(tf.multiply(output, action), reduction_indices = 1)
print(qreward)
loss = tf.reduce_mean(tf.square(reward - qreward))

#optimizer = tf.train.AdamOptimizer().minimize(loss)
optimizer = tf.train.RMSPropOptimizer(0.001).minimize(loss)




Tensor("conv3/Relu:0", shape=(?, 7, 5, 32), dtype=float32)
Tensor("Reshape:0", shape=(?, 1120), dtype=float32)
Tensor("Relu:0", shape=(?, 4), dtype=float32)
Tensor("Sum:0", shape=(?,), dtype=float32)


In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


In [8]:

def get_random_hist(history, ncount): 
    minibatch = random.sample(history, ncount)
    state_s = [val[0] for val in minibatch]
    reward = [val[2] for val in minibatch]
    state_s1 = [val[3] for val in minibatch]
    
    actions = []
    for index, action in enumerate([val[1] for val in minibatch]):
        actionlist = [0]*ACTIONCOUNT
        actionlist[action]=1
        actions.append(actionlist)
        
    return state_s, actions, reward, state_s1
    

In [9]:
avg_rewards = []
avg_estimated = []

In [10]:
n_epochs = 200000
batch_size = 32
epsilon = 1.00
REPLAY_MEMORY_LENGTH = 10000 # use a replay memory of one million most recent frames
REPLAY_MEMORY_STRETCH = 500
EPOCH_SIZE = 500 #  One epoch corresponds to 50000 minibatch weight updates or roughly 30 minutes of training time.
EPOCH_COUNT = 1000
ANNEAL_IN_N_STEPS = 1000000
lastframes = [x[0] for x in history[-FRAMES_TO_CONSIDER:]]

for epoch_i in range(EPOCH_COUNT):
    sumrewards = 0.0
    sumestimatedfuture = 0.0
    for _ in range(EPOCH_SIZE): 
        states, actions, rewards, newstates = get_random_hist(state_rewards, batch_size)
        
        estimatedrewards = sess.run(output, feed_dict={inputimage:newstates})
        estimatedrewards = np.amax(estimatedrewards,axis=1)
        values = []
        for index in range(batch_size):
            values.append(rewards[index] + estimatedrewards[index])
        
        _, l = sess.run([optimizer, loss], feed_dict={inputimage:states, reward: values, action: actions})
        
        newimage = frames_to_nd_image(lastframes)
        if random.random() <= epsilon:
            newaction = env.action_space.sample() 
        else:
            actions = sess.run(output, feed_dict={inputimage:np.array([newimage], dtype=np.float32)})
            actions = actions[0]
            newaction = np.argmax(np.array(actions))

        epsilon -= 0.9 / ANNEAL_IN_N_STEPS 
        if epsilon < 0.1:
            epsilon = 0.1
        newobservation, rewardhere, done, info = env.step(newaction)
        frame = env.render(mode = 'rgb_array')
      
        observation_gray = rgb2gray(np.array(frame))
        lastframes.append(observation_gray)
        lastframes = lastframes[1:]

        newerimage = frames_to_nd_image(lastframes)
        
        state_rewards.append((newimage,newaction, rewardhere, newerimage))
        
        estimatedfuturereward = max(max(sess.run(output, feed_dict={inputimage:np.array([newerimage])})))
        sumrewards += rewardhere
        sumestimatedfuture += estimatedfuturereward
        
        
        if len(state_rewards) > REPLAY_MEMORY_LENGTH + REPLAY_MEMORY_STRETCH: 
            state_rewards = state_rewards[REPLAY_MEMORY_LENGTH:]

        if done:
            observation = env.reset()
    timeup = strftime("%Y-%m-%d %H:%M:%S", gmtime())
    avg_reward_epoch = sumrewards/EPOCH_SIZE
    avg_estimated_epoch = sumestimatedfuture/EPOCH_SIZE
    avg_rewards.append(avg_reward_epoch)
    avg_estimated.append(avg_estimated_epoch)
    print("%s: Epoch: %d, avg reward: %f, avg estimated: %f, estimated future: %f, loss: %f, trainmat: %d" % (timeup, epoch_i,avg_reward_epoch , avg_estimated_epoch , estimatedfuturereward, l, len(state_rewards)))
        
        

2017-06-30 12:11:36: Epoch: 0, avg reward: 0.008000, avg estimated: 0.878344, estimated future: 0.999999, loss: 0.000000, trainmat: 600
2017-06-30 12:12:00: Epoch: 1, avg reward: 0.004000, avg estimated: 0.999999, estimated future: 0.999999, loss: 0.031250, trainmat: 1100
2017-06-30 12:12:24: Epoch: 2, avg reward: 0.006000, avg estimated: 1.000000, estimated future: 1.000000, loss: 0.031250, trainmat: 1600
2017-06-30 12:12:49: Epoch: 3, avg reward: 0.006000, avg estimated: 1.000000, estimated future: 1.000000, loss: 0.000000, trainmat: 2100


KeyboardInterrupt: 

In [ ]:
whoopframes = []
for epoch_i in range(1000):
    if random.random() <= 0.2:
        newaction = env.action_space.sample() 
    else:
        newimage = frames_to_nd_image(lastframes)
        actions = sess.run(output, feed_dict={inputimage:np.array([newimage], dtype=np.float32)})
        actions = actions[0]
        newaction = np.argmax(np.array(actions))

        
    newobservation, rewardhere, done, info = env.step(newaction)
    frame = env.render(mode = 'rgb_array')
    whoopframes.append(frame)
    
    observation_gray = rgb2gray(np.array(frame))
    lastframes.append(observation_gray)
    lastframes = lastframes[1:]

    newerimage = frames_to_nd_image(lastframes)
    estimatedfuturereward = max(max(sess.run(output, feed_dict={inputimage:np.array([newerimage])})))
    
    wanttoadd = (newimage, newaction, rewardhere+estimatedfuturereward)
#    state_rewards

    state
    if done:
        observation = env.reset()

display_frames_as_gif(whoopframes)        

In [ ]:
def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    if len(frames) == 0:
        return
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    anim.save('animation.gif', writer = 'imagemagick', fps=30)

    display(display_animation(anim, default_mode='loop'))
    

display_frames_as_gif(whoopframes[100:250])        

In [ ]:
print(wanttoadd[0].shape)
print(wanttoadd[1])
print(wanttoadd[2])
print(state_rewards[0][0].shape)
print(state_rewards[0][1])
print(state_rewards[0][2])

In [ ]:
% matplotlib inline
plt.ioff()
plt.imshow(lastframes[3])
plt.show()
print(lastframes[2].shape)

frame = env.render(mode = 'rgb_array')
im.set_data(frame)

observation_gray = rgb2gray(np.array(frame))
print(observation_gray.shape)
history[0][0].shape

In [ ]:

print(inputimage)
print(np.array([newimage]).shape)
print(np.array([newimage],dtype=np.float32).dtype,)
print(output)
actions = sess.run(output, feed_dict={inputimage:np.array([newimage], dtype=np.float32)})
print(actions)

In [ ]:

def show_episode(env, parameters):  
    observation = env.reset()
    firstframe = env.render(mode = 'rgb_array')
    frames = [firstframe]
    
    for _ in xrange(10500):
        action = env.action_space.sample() 

        observation, reward, done, info = env.step(action)
        frame = env.render(mode = 'rgb_array')
        frames.append(frame)
        if done:
            break
    return frames

frames = show_episode(env, bestparams)
display_frames_as_gif(frames)

### Conclusion
Congratulations! You made your first autonomous pole-balancer in the OpenAI gym environment. Now that this works it is time to either improve your algorithm, or start playing around with different environments. 

### Acknowledgments 
This blogpost is the first part of my TRADR summerschool workshop on using human input in reinforcement learning algorithms. More information can be found [on their homepage](https://sites.google.com/view/tradr/home)
